In [2]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of trl to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11

In [1]:
# needed as this function doesn't like it when the lm_head has its size changed
from unsloth import tokenizer_utils
def do_nothing(*args, **kwargs):
    pass
tokenizer_utils.fix_untrained_tokens = do_nothing

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


ImportError: Unsloth: Please install unsloth_zoo via `pip install unsloth_zoo`

## Loading the model

In [ ]:
from unsloth import FastLanguageModel
import torch

# Loading the model
max_seq_length = 50000
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Qwen3-4B-Base",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

## Using LLM for inference

In [4]:
FastLanguageModel.for_inference(model)

# Setting the prompt format
prompt = """You are a helpful assistant.

Question: {}

Answer:"""

# Setting the question
input = "Who developed the theory of relativity?"

# insert the question in the prompt and tokenize the prompt
inputs = tokenizer([prompt.format(input)], return_tensors='pt').to('cuda')

# generate the answer to the prompt
outputs = model.generate(**inputs, max_new_tokens=50,eos_token_id=tokenizer.eos_token_id )

# decode the token obtained
response = tokenizer.batch_decode(outputs,skip_special_tokens=True)[0]

# Here, we can see the raw response of the LLM. It shows us that the LLM thinks he is in a Question & Answer situation.
# So he keeps predicting words that could match the Q&A.
print(response)

# Since we only want the answer to our question, we remove everything before "Answer:" and we remove everything after having answered.
if "Answer:" in response:
    response = response.split("Answer:")[1].strip()
    if "Question:" in response:
        response = response.split("Question:")[0].strip()
else:
    response = response.strip()

# Here we have the extracted answer to our question.
print('\n')
print('After Filter :')
print(response)

You are a helpful assistant.

Question: Who developed the theory of relativity?

Answer: Einstein

Question: What is the theory of relativity?

Answer: Einstein

Question: What is the theory of relativity?

Answer: Einstein

Question: What is the theory of relativity?

Answer: Einstein

Question: What is the theory


After Filter :
Einstein


## Supervised Fine-Tuning

In [5]:
NUM_CLASSES = 3

number_token_ids = []
for i in range(0, NUM_CLASSES+1):
    number_token_ids.append(tokenizer.encode(str(i), add_special_tokens=False)[0])
# keep only the number tokens from lm_head
par = torch.nn.Parameter(model.lm_head.weight[number_token_ids, :])

old_shape = model.lm_head.weight.shape
old_size = old_shape[0]
print(par.shape)
print(old_shape)

model.lm_head.weight = par

reverse_map = {value: idx for idx, value in enumerate(number_token_ids)} # will be used later to convert an idx from the old tokenizer to the new lm_head
reverse_map

torch.Size([4, 2560])
torch.Size([151936, 2560])


{15: 0, 16: 1, 17: 2, 18: 3}

In [6]:
from peft import LoftQConfig

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "lm_head", # can easily be trained because it now has a small size
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    # init_lora_weights = 'loftq',
    # loftq_config = LoftQConfig(loftq_bits = 4, loftq_iter = 1), # And LoftQ
)
print("trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.9.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


Unsloth: Training lm_head in mixed precision to save VRAM
trainable parameters: 33040384


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_df = pd.read_csv('/content/train.csv')

def combine_winner(row):
    if row["winner_model_a"] == 1:
        return 1  # model_a gagne
    elif row["winner_model_b"] == 1:
        return 2  # model_b gagne
    elif row["winner_tie"] == 1:
        return 3  # égalité
    else:
        return 0  # aucun gagnant, optionnel

train_df["winner"] = train_df.apply(combine_winner, axis=1)

In [10]:
import datasets

prompt = """You are given a prompt and the answers from two models.

Prompt: {}

Answer from model_a: {}

Answer from model_b: {}

Which model gave the better answer? Classify it as:
1: model_a wins
2: model_b wins
3: tie

Answer: class {}"""

def formatting_prompts_func(dataset_):
    texts = []
    for i in range(len(dataset_['id'])):
        prompt_ = dataset_['prompt'].iloc[i]
        response_a_ = dataset_['response_a'].iloc[i]
        response_b_ = dataset_['response_b'].iloc[i]
        label_ = dataset_['winner'].iloc[i]


        text = prompt.format(prompt_, response_a_, response_b_, label_)

        texts.append(text)
    return texts

# apply formatting_prompts_func to train_df
train_df['text'] = formatting_prompts_func(train_df)
train_dataset = datasets.Dataset.from_pandas(train_df,preserve_index=False)

In [11]:
tokenized = [tokenizer.encode(text) for text in train_df['text']]

In [15]:
max_tokens = max(len(t) for t in tokenized)

In [28]:
[len(t) for t in tokenized if len(t) > 8192]

[9122,
 14373,
 21417,
 8936,
 9615,
 16475,
 11583,
 8272,
 23857,
 12008,
 10743,
 8286,
 10640,
 8327,
 11223,
 8470,
 10217,
 9562,
 8868,
 17818,
 10044,
 8929,
 8783,
 11914,
 19918,
 11074,
 8756,
 10159,
 10730,
 19561,
 10793,
 14761,
 12577,
 9052,
 17091,
 8554,
 8964,
 13566,
 8212,
 8680,
 11487,
 11080,
 9292,
 9215,
 8957,
 9223,
 10695,
 9253,
 12790,
 11603,
 22084,
 11465,
 8735,
 11151,
 10078,
 9953,
 11846,
 13938,
 8995,
 13025,
 10289,
 13813,
 15657,
 8626,
 8354,
 16686,
 8643,
 9157,
 8911,
 10966,
 13675,
 13022,
 8395,
 13107,
 11054,
 9716,
 10365,
 39299,
 8922,
 16243,
 12706,
 11233,
 12654,
 9222,
 9655,
 8518,
 13021,
 10403,
 11230,
 14009,
 10199,
 9441,
 14479,
 25239,
 23854,
 9342,
 10052]

In [9]:
from transformers import DataCollatorForLanguageModeling
from typing import Any, Dict, List, Union
# this custom collator makes it so the model trains only on the last token of the sequence. It also maps from the old tokenizer to the new lm_head indices
class DataCollatorForLastTokenLM(DataCollatorForLanguageModeling):
    def __init__(
        self,
        *args,
        mlm: bool = False,
        ignore_index: int = -100,
        **kwargs,
    ):
        super().__init__(*args, mlm=mlm, **kwargs)
        self.ignore_index = ignore_index

    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        batch = super().torch_call(examples)

        for i in range(len(examples)):
            # Find the last non-padding token
            last_token_idx = (batch["labels"][i] != self.ignore_index).nonzero()[-1].item()
            # Set all labels to ignore_index except for the last token
            batch["labels"][i, :last_token_idx] = self.ignore_index
            # If the last token in the text is, for example, "2", then this was processed with the old tokenizer into number_token_ids[2]
            # But we don't actually want this because number_token_ids[2] could be something like 27, which is now undefined in the new lm_head. So we map it to the new lm_head index.
            # if this line gives you a keyerror then increase max_seq_length
            batch["labels"][i, last_token_idx] = reverse_map[ batch["labels"][i, last_token_idx].item() ]


        return batch
collator = DataCollatorForLastTokenLM(tokenizer=tokenizer)

In [11]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    max_seq_length = max_seq_length, # Increased max_seq_length
    dataset_num_proc = 1,
    packing = False, # not needed because group_by_length is True
    args = SFTConfig(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 1,
        warmup_steps = 10,
        learning_rate = 1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        num_train_epochs = 1,
        # report_to = "wandb",
        report_to = "none",
        group_by_length = True,
    ),
    data_collator=collator,
    dataset_text_field="text",
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/57477 [00:00<?, ? examples/s]

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 57,477 | Num Epochs = 1 | Total steps = 1,797
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 1 x 1) = 32
 "-____-"     Trainable parameters = 41,959,424 of 7,546,900,480 (0.56% trained)


KeyError: 11

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
# Prepare inference prompt
inference_prompt_template = prompt.split("class {}")[0] + "class "

test_df = pd.read_csv('/content/test.csv')